In [1]:
# %load_ext autoreload 
# %autoreload 2

In [1]:
import time
start_time = time.time()

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, StructField, StructType, StringType, IntegerType, TimestampType
from pyspark.sql import functions as F

import pandas as pd
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
print('kya hai')
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import count, avg
from matplotlib.pyplot import scatter
from pyspark.sql import functions as F

from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, StructField, StructType, StringType, IntegerType, TimestampType
#from pyspark.sql import functions as F
from pyspark.sql import functions as f

#from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.sql.functions import countDistinct
#from pyspark.ml.linalg import Vectors, SparseVector
#from pyspark.ml.clustering import LDA
#from pyspark.ml.functions import vector_to_array

import datetime
from  pyspark.sql.functions import abs
from  pyspark.sql.functions import col

from pyspark.sql.functions import desc
from pyspark.sql.functions import desc
import numpy as np
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql import DataFrameStatFunctions as stat
import numpy as np
import pandas as pd
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql.types import *
from pyspark.sql import Window
from pyspark.sql import SparkSession

import requests
from collections import defaultdict
from pymongo import MongoClient
from tqdm import tqdm
from pyspark.sql import DataFrameStatFunctions as stat
from pyspark.sql import Window
import pyspark.sql.functions as F
import os
import time
import json
import pickle
from dateutil.parser import parse
import seaborn as sns
from pyspark.sql.functions import explode


NIS_DATA_BASE_PATH = "gs://nis-segment-datasource-v3/processed/"
NIS_OLD_DATA_BASE_PATH = "gs://nis-localytics-datasource/processed/"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = SparkConf().setAll([('spark.sql.broadcastTimeout',1000)
    # ('spark.driver.memory', '20g'), \
#                            ('spark.broadcast.blockSize', '10m'),\
#                            ('spark.dynamicAllocation.enabled','False'),\
#                            ('spark.executor.instances','9999')
# #                            ,("spark.sql.autoBroadcastJoinThreshold",-1)
                          ])
sc = SparkContext.getOrCreate(conf=conf)
#sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
#sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


kya hai
time: 15.7 ms (started: 2023-02-19 07:49:33 +00:00)


In [4]:
def to_disk(a,path):
    with open(path, 'wb') as handle:
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
def from_disk(path):
    with open(path, 'rb') as handle:
        b = pickle.load(handle)
    return b

def get_path(dates, prefix, padding=None):
    dates_ = dates + []
    if padding:
        st_date, ed_date = sorted(dates)[0], sorted(dates)[-1]
        for i in range(1, 4):
            d = (datetime.datetime.strptime(ed_date, date_fmt) + datetime.timedelta(days=i)).strftime(date_fmt)
            if d < datetime.datetime.today().strftime(date_fmt):
                dates_.append(d)
    dates_ = list(set(dates_) - set(["2019/02/17", "2019/02/18", "2019/05/28", "2019/06/03", "2019/07/02", "2019/07/03", "2019/07/04", "2019/11/13", "2019/11/14", "2020/02/22", "2020/03/31", "2020/04/16", "2020/04/18", "2020/05/11", "2021/05/13"]))
    paths = []
    for date in dates_:
        base_path = NIS_DATA_BASE_PATH
        if date < "2018/06/26":
            base_path = NIS_OLD_DATA_BASE_PATH
        paths.append(base_path + date + "/" + prefix + "/*.parquet")
    return paths

date_fmt = "%Y/%m/%d"
month_fmt = "%Y/%m"

def millis2date(x):
    try:
        if x < 15000000000:
            return datetime.datetime.fromtimestamp(x).strftime(date_fmt)
        else:
            return datetime.datetime.fromtimestamp(x / 1000.).strftime(date_fmt)
    except:
        return "1970/01/01"

def millis2month(x):
    try:
        if x < 15000000000:
            return datetime.datetime.fromtimestamp(x).strftime(month_fmt)
        else:
            return datetime.datetime.fromtimestamp(x / 1000.).strftime(month_fmt)
    except:
        return "1970/01"

millis2date_udf = F.udf(millis2date, StringType())
millis2month_udf = F.udf(millis2month, StringType())

def divide_maps(d1, d2):
    keys = set(d1.keys()).intersection(set(d2.keys()))
    res = {}
    for k in keys:
        res[k] = d1[k] * 1. / (d2[k] + 1e-10)
    return res

def timediff(y, x, date_fmt="%Y/%m/%d"): 
    end = datetime.datetime.strptime(y, date_fmt)
    start = datetime.datetime.strptime(x, date_fmt)
    delta = (end - start).days
    return delta

def monthdiff(y, x, month_fmt="%Y/%m"): 
    millis = y - x
    delta = millis / (1000 * 3600 * 24 * 30)
    return delta

timediff_udf = udf(timediff, IntegerType())
monthdiff_udf = udf(monthdiff, IntegerType())

def filter_platform(data, platform=None):
    if platform == "ANDROID":
        data = data.filter(data.platform == "ANDROID")
    elif platform == "IOS":
        data = data.filter(data.platform != "ANDROID")
    return data

def filter_category(data, categories=None):
    if categories:
        data = data.filter(data.categoryWhenEventHappened.isin(categories))
    return data

def filter_tenant(data, tenant=None):
    if tenant in ['hi', 'HINDI']:
        data = data.filter(data.tenant.isin(['hi', 'HINDI', 'Hindi', 'hindi']))
    elif tenant in ['en', 'ENGLISH']:
        data = data.filter(~data.tenant.isin(['hi', 'HINDI', 'Hindi', 'hindi']))
    return data


def filter_app(data, app_name=None):
    if 'appName' in data.columns:
        if app_name:
            data = data.filter(data.appName == app_name)
        else:
            data = data.filter((data.appName != "mini") & (data.appName != "crux"))
    return data


#### Hash definition
def InternalSeqHash(deviceId):
    h = 0
    for c in list(deviceId):
        h = (31*h + ord(c)) & 0xFFFFFFFF
    return ((h + 0x80000000) & 0xFFFFFFFF) - 0x80000000

def hashIdentifier(deviceId, M=100):
    hId = (abs(InternalSeqHash(deviceId)) % M) + 1
    return hId


def date_range_generator(start_date,n_days):
    date_fmt = "%Y/%m/%d"
    sign = (-1 if n_days<0 else 1)
    n_days = abs(n_days)
    st_date = parse(start_date)
    return [(st_date + datetime.timedelta(days=sign*i)).strftime(date_fmt) for i in range(n_days)]


import matplotlib.pyplot as plt






from pyspark.sql.functions import countDistinct
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.sql.functions import countDistinct
#from pyspark.ml.liqnalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
from pyspark.ml.functions import vector_to_array

from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.sql.functions import countDistinct
#from pyspark.ml.liqnalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
from pyspark.ml.functions import vector_to_array

# Content data for news hash ids

from pymongo import MongoClient
import datetime

def getColl(hosts,db_name,coll_name):
    hosts = ",".join(hosts)
    url = "mongodb://root:superman@"+hosts+"/admin"
    client = MongoClient(url,serverSelectionTimeoutMS=8000)
    coll = client[db_name][coll_name]
    return coll


start_dt_time = datetime.datetime(2023, 1, 11)


hosts = ["172.16.11.196", "172.16.11.195", "172.16.11.194"]
db_name = "nis-news"
coll_news = getColl(hosts,db_name,'News')


#start_dt_time= datetime.datetime.now() - datetime.timedelta(days=163)

print(start_dt_time)

all_news = coll_news.find({"createdAt":{"$gt":start_dt_time}},{'_id':1,"categories":1,"createdAt":1,"internalCategories":1,"title":1,"content":1})

#merging content and hash ids

from pandas import DataFrame

from pymongo import MongoClient
import datetime
#DataFrame[deviceId: string, overallTimeSpent: double, hashId: string, eventTimestamp: bigint]
#keep same date for dataframe
def data_content(df_,st_date=datetime.datetime(2022, 12, 13)):
    def getColl(hosts,db_name,coll_name):
        hosts = ",".join(hosts)
        url = "mongodb://root:superman@"+hosts+"/admin"
        client = MongoClient(url,serverSelectionTimeoutMS=8000)
        coll = client[db_name][coll_name]
        return coll


    start_dt_time = st_date


    hosts = ["172.16.11.196", "172.16.11.195", "172.16.11.194"]
    db_name = "nis-news"
    coll_news = getColl(hosts,db_name,'News')


    #start_dt_time= datetime.datetime.now() - datetime.timedelta(days=163)

    print(start_dt_time)

    all_news = coll_news.find({"createdAt":{"$gt":start_dt_time}},{'_id':1,"categories":1,"createdAt":1,"internalCategories":1,"title":1,"content":1})
    
    list_cur = list(all_news)
    df_content = DataFrame(list_cur)
    DF_cont=spark.createDataFrame(df_content[['content','title','_id']])#.cache() 
    DF_final=df_.join(DF_cont,df_.hashId ==  DF_cont._id,"inner").select('deviceId','hashId','overallTimeSpent','content','title')
    
    return DF_final
    


from spacy.lang.en.stop_words import STOP_WORDS
stopwords=STOP_WORDS

import string
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

def tokenization(text):
    tokens = text.split()
    return tokens

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree


def preprocess(text):
    text = remove_punctuation(text)

    text = text.lower()

    tokens = tokenization(text)
    tokens = remove_stopwords(tokens)
    
    
    return tokens

from pyspark.sql.functions import udf


prepr_udf = F.udf(preprocess,ArrayType(StringType()))


def merge_list(list1,list2):
    lis=list1+list2
    return lis
merge_udf=F.udf(merge_list,ArrayType(StringType()))

2023-01-11 00:00:00


In [ ]:
st_date = datetime.datetime(2023, 2, 11)

date_fmt = "%Y/%m/%d"
n_days = 0


dates = [(st_date + datetime.timedelta(days=i)).strftime(date_fmt) for i in range(n_days+1)]
#print('dates:',dates)


#timespent data
paths = get_path(dates, 'timeSpentFrontEvents')
#paths = get_path(dates, 'otherEvents')
data_timespent = (sqlContext.read.parquet(*paths).select("deviceId","overallTimeSpent","hashId","eventTimestamp","categoryWhenEventHappened","eventName")
.filter("categoryWhenEventHappened='My Feed' and eventName='TimeSpent-Front' and overallTimeSpent<58")
.select("deviceId","overallTimeSpent","hashId","eventTimestamp")
                 )

st_date = datetime.datetime(2023, 1, 11)
n_days = 0

dates = [(st_date + datetime.timedelta(days=i)).strftime(date_fmt) for i in range(n_days+1)]
#print('dates:',dates)


#timespent data
paths = get_path(dates, 'timeSpentFrontEvents')
data_dec = (sqlContext.read.parquet(*paths)
.select("deviceId","overallTimeSpent","hashId","eventTimestamp","categoryWhenEventHappened","eventName")
.filter("categoryWhenEventHappened='My Feed' and eventName='TimeSpent-Front' and overallTimeSpent<58").select("deviceId")
            #major change#
            .distinct()
)


Data_LDA=data_timespent.join(data_dec,['deviceId'],"inner")


#tmp_loc="gs://pvtrough_asia_south1/tmp/gourav_model"
tmp_loc="/home/Gourav/Set_B"
(data_content(Data_LDA,st_date=datetime.datetime(2023, 2, 11))
.filter("title is Not Null and content is Not Null")
.withColumn("Preprocess_title", prepr_udf("title"))
.withColumn("Preprocess_content", prepr_udf("content"))
.withColumn("feature", merge_udf("Preprocess_title","Preprocess_content"))          
.select('deviceId','hashId','overallTimeSpent','feature')          
         ).write.parquet(tmp_loc)



In [6]:
from Utils import *
def getNewsData(d1, d2):
    newsMap = getNewsInDates(d1, d2)
    hashIdList = list(newsMap.keys())

    hashIdsWithFilter = []
    for h in hashIdList:
        if 'newsLanguage' in newsMap[h] and newsMap[h]['newsLanguage'] == 'english' and newsMap[h]['publishGroupList'][0]['countryCode'] == 'IN':
            hashIdsWithFilter.append(h.split('-')[0])
    
    return hashIdsWithFilter, newsMap


In [7]:
import logging

logger = logging.getLogger(str(datetime.datetime.today().date()))
hdlr = logging.FileHandler(
    'vectorize-logs/' + str(datetime.datetime.today().date()) + '.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.DEBUG)

In [8]:
def Log(s, flag=True):
    if flag:
        logger.info(s)
        print(s)

Log('----', flag=True)


----


In [9]:
n_days = 40
st_date = datetime.datetime.now() - datetime.timedelta(days=n_days+1)

date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]

# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

hashIdsWithFilter, newsMap = getNewsData(dates[0], datetime.datetime.now())

In [10]:
NIS_RAW_DATA_BASE_PATH = "gs://inshorts-segment-raw/data/segment-raw-v5/"

path = get_raw_path(datetime.datetime.now().strftime("%Y/%m/%d"))
today_data = process_raw_data(path)
today_data = today_data.filter(today_data.hashId.isin(hashIdsWithFilter))

In [11]:
datestr = [d.strftime(date_fmt) for d in dates]
paths = get_path(datestr, 'timeSpentFrontEvents')

data = sqlContext.read.parquet(*paths)

data = filter_app(data, app_name=None)
data = filter_tenant(data, tenant='en')

# data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))
data = data.select(data.deviceId, data.overallTimeSpent, (F.split(data.hashId, '-')[0]).alias('hashId'))\
        .groupby('deviceId', 'hashId') \
        .agg(F.max('overallTimeSpent').alias('overallTimeSpent'))

data = data.filter(data.hashId.isin(hashIdsWithFilter))

In [12]:
data = data.union(today_data)
data.cache()

DataFrame[deviceId: string, hashId: string, overallTimeSpent: double]

In [13]:
def popularity(data, newsMeanMap):
    Log("Computing news mean")
    newsMeandf = data.groupby(data.hashId).agg({'overallTimeSpent' : 'sum', 'hashId' : 'count'}).toPandas()

    for v in newsMeandf.values:
        newsMeanMap[v[0]] = newsMeanMap[v[0]][0] + v[1],  newsMeanMap[v[0]][1] + v[2]

newsMeanMap = defaultdict(lambda: (7, 1))
popularity(data, newsMeanMap)

Computing news mean


In [14]:
# import re
import string
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')

#defining the object for stemming
porter_stemmer = PorterStemmer()

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

def tokenization(text):
    tokens = text.split()
    return tokens

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

def preprocess(text):
    text = remove_punctuation(text)

    text = text.lower()

    tokens = tokenization(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    tokens = lemmatizer(tokens)
    
    return tokens

newsMapProcessed = {}
for hId in tqdm(newsMap):
    h = hId.split('-')[0]
    newsMapProcessed[h] = {}
    newsMapProcessed[h]['title'] = preprocess(newsMap[hId]['title'])
    newsMapProcessed[h]['content'] = preprocess(newsMap[hId]['content'])
    
    newsMapProcessed[h]['features'] = newsMapProcessed[h]['title'] + newsMapProcessed[h]['content']


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/anunayarunav/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/anunayarunav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/anunayarunav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|████████████████████████████████████████████| 23281/23281 [00:32<00:00, 726.07it/s]


In [ ]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA
import time

training_start = time.time()

def newsToTokens(hashId, overallTimeSpent):
    tokens = newsMapProcessed[hashId]['features']
    #count = int(min(100, overallTimeSpent)/8) + 1
    return tokens

def toTokenCollection(tokensList):
    arr = []
    for tokens in tokensList:
        for token in tokens:
            arr.append(token)
    
    return arr

newsToTokensUdf = F.udf(newsToTokens, ArrayType(StringType()))

data_filtered = data.filter(F.udf(lambda hashId, overallTimeSpent: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))
data_text = data_filtered.select('deviceId', newsToTokensUdf('hashId', 'overallTimeSpent').alias('tokens')).groupby('deviceId').agg(F.collect_list('tokens').alias('tokensAll'))
data_title = data_text.select('deviceId', F.udf(toTokenCollection, ArrayType(StringType()))('tokensAll').alias('tokensList'))
row = data_filtered.take(1)

cv = CountVectorizer()
cv.setInputCol('tokensList')
cv.setOutputCol("vectors")

model = cv.fit(data_title)

data_vector = model.transform(data_title)

numClusters = 10
lda = LDA(k=numClusters, seed=1, optimizer="online")
lda.setFeaturesCol("vectors")

ldaModel = lda.fit(data_vector)

data_lda = ldaModel.transform(data_vector)

topicDf = data_lda.select('deviceId', 'topicDistribution').toPandas()

labels = {}

for v in tqdm(topicDf.values):
    labels[v[0]] = int(np.argmax(np.array(v[1])))

Log("Training took %s minutes"%(int((time.time() - training_start) / 60)))

In [16]:
from ColabUtils import *

deviceVectors = {}
for v in tqdm(topicDf.values):
    deviceVectors[v[0]] = [float(i) for i in v[1]]
insertDeviceVectorsInMongo(deviceVectors)

  6%|██▎                                     | 178829/3024556 [02:23<26:49, 1768.59it/s]22/03/09 09:21:42 WARN org.apache.spark.network.server.TransportChannelHandler: Exception in connection from /171.16.11.158:52640
java.io.IOException: Connection timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:253)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1133)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:350)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:148)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:714)
	at io.netty.channel.nio.NioEventLoop.proce

In [17]:
newClusterWiseDevices = defaultdict(lambda: set())
for deviceId, cluster in tqdm(labels.items()):
    newClusterWiseDevices[cluster].add(deviceId)

100%|████████████████████████████████████| 3024556/3024556 [00:01<00:00, 1513935.48it/s]


In [1]:
from ColabUtils import *
deviceClusterMapExisiting = getDeviceClustersFromMongo([])

13012497

In [19]:
clusterWiseDevices = defaultdict(lambda: set())
for d in tqdm(deviceClusterMapExisiting):
    clusterWiseDevices[deviceClusterMapExisiting[d]].add(d)

100%|█████████████████████████████████████| 6868334/6868334 [00:06<00:00, 982225.95it/s]


In [20]:
# [len(newClusterWiseDevices[x]) for x in newClusterWiseDevices]

In [21]:
marked = {}
clusterMap = {}
for x in newClusterWiseDevices:
    mxinter = -1
    mxidx = -1
    for y in clusterWiseDevices:
        if y in marked:
            continue
        inter = round(len(clusterWiseDevices[y].intersection(newClusterWiseDevices[x])) / len(clusterWiseDevices[y])*100, 3)
        if inter > mxinter:
            mxinter = inter
            mxidx = y
            
    if mxidx < 0:
        mxidx = x
    
    marked[mxidx] = 1
    clusterMap[x] = mxidx
    Log("Mapping cluster " + str(x) + " to " + str(mxidx) + ", %age intersection = " + str(mxinter))

Mapping cluster 5 to 4, %age intersection = 10.925
Mapping cluster 4 to 5, %age intersection = 30.256
Mapping cluster 7 to 2, %age intersection = 18.144
Mapping cluster 3 to 0, %age intersection = 23.84
Mapping cluster 1 to 1, %age intersection = 14.43
Mapping cluster 9 to 7, %age intersection = 2.171
Mapping cluster 0 to 6, %age intersection = 13.46
Mapping cluster 2 to 8, %age intersection = 9.854
Mapping cluster 6 to 9, %age intersection = 0.596
Mapping cluster 8 to 3, %age intersection = 6.181


In [22]:
updatedDeviceClusters = {}
for cluster in newClusterWiseDevices:
    for deviceId in tqdm(newClusterWiseDevices[cluster]):
        updatedDeviceClusters[deviceId] = clusterMap[cluster]

100%|██████████████████████████████████████| 126944/126944 [00:00<00:00, 1031048.56it/s]


In [23]:
insert_start = time.time()
from ColabUtils import *
insertDeviceClustersInMongo(updatedDeviceClusters, deviceClusterMapExisiting, Log)

Inserted 100000 Device Clusters
Inserted 200000 Device Clusters
Inserted 300000 Device Clusters
Inserted 400000 Device Clusters
Inserted 500000 Device Clusters
Inserted 600000 Device Clusters
Inserted 700000 Device Clusters
Inserted 800000 Device Clusters
Inserted 900000 Device Clusters
Inserted 1000000 Device Clusters
Inserted 1100000 Device Clusters
Inserted 1200000 Device Clusters
Inserted 1300000 Device Clusters
Inserted 1400000 Device Clusters
Inserted 1500000 Device Clusters
Inserted 1600000 Device Clusters
Inserted 1700000 Device Clusters
Inserted 1800000 Device Clusters
Inserted 1900000 Device Clusters
Inserted 2000000 Device Clusters
Inserted 2100000 Device Clusters
Updated 71.1 device clusters


In [24]:
insert_time = (time.time() - insert_start)

Log("Insertion took %s percent of time" % round(100 * insert_time / (time.time() - start_time), 3))

Insertion took 8.455 percent of time


In [25]:
dataWithClusters = data_filtered.select('hashId', 'overallTimeSpent', F.udf(lambda deviceId: updatedDeviceClusters[deviceId], IntegerType())('deviceId').alias('cluster'))
newsClusterMean = dataWithClusters.groupby('hashId', 'cluster').agg(F.sum('overallTimeSpent'),F.count('hashId'))
newsClusterMeanDf = newsClusterMean.toPandas()

22/03/09 09:33:36 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1058.0 KiB
22/03/09 09:34:34 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1056.1 KiB


In [26]:
newsTSpentMap = defaultdict(lambda: defaultdict(lambda: (7, 1)))
for v in tqdm(newsClusterMeanDf.values):
    newsTSpentMap[v[0]][v[1]] = (v[2], v[3])

100%|███████████████████████████████████████| 125570/125570 [00:00<00:00, 567297.85it/s]


In [27]:
Log("Inserting News Score Data")
insert_start = time.time()
insertNewsTSpentInMongo(newsTSpentMap)
insertNewsScoreInMongo(newsTSpentMap)
insert_time = (time.time() - insert_start)
Log("Inserting News Score took %s percent of time" % round(100 * insert_time / (time.time() - start_time), 3))

Inserting News Score Data
Inserting News Score took 0.946 percent of time


In [28]:
Log("Exectution finished successfully, in : %s minutes, %s seconds " % (int((time.time() - start_time) / 60), int((time.time() - start_time) % 60)), flag=True)

Exectution finished successfully, in : 54 minutes, 20 seconds 
